This is a practice run at a Kagglecompetition for predicting if a Titanic passegner 

In [237]:
import pandas as pd
import tensorflow as tf

In [238]:
df = pd.read_csv("data/train.csv")

In [239]:
df.head()
#Note - Survived 0 means died, Survived 1 means lived
#SibSp means how many siblings on the ship
#Parch means how many parents were on the ship
#embarked means C = Cherbourg, Q = Queenstown, S = Southampton

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [240]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df = df.dropna()

In [241]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [242]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [243]:
cols_to_norm = ['Fare']

In [244]:
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x-x.min()) / (x.max() - x.min()))

In [245]:
df['Pclass'] = pd.to_numeric(df['Pclass'])

In [246]:
type(df['Pclass'][0])

numpy.int64

In [247]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,0.014151,S
1,1,1,female,38.0,1,0,0.139136,C
2,1,3,female,26.0,0,0,0.015469,S
3,1,1,female,35.0,1,0,0.103644,S
4,0,3,male,35.0,0,0,0.015713,S


In [248]:
#p_class = tf.feature_column.categorical_column_with_hash_bucket('Pclass',hash_bucket_size=3)
p_class = tf.feature_column.numeric_column('Pclass')
sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex',['male','female'])
age = tf.feature_column.numeric_column('Age')
p_arch = tf.feature_column.categorical_column_with_hash_bucket('Parch',hash_bucket_size=4)
fare = tf.feature_column.numeric_column('Fare')
embarked = tf.feature_column.categorical_column_with_vocabulary_list('Embarked',['C','S','Q'])

# embedded_p_class = tf.feature_column.embedding_column(p_class,dimension=3)
bucket_age = tf.feature_column.bucketized_column(age,boundaries=[18,25,30,35,40,56])
embedded_sex = tf.feature_column.embedding_column(sex,dimension=2)
embedded_p_arch = tf.feature_column.embedding_column(p_arch,dimension=4)
embedded_embarked = tf.feature_column.embedding_column(embarked,dimension=3)

In [249]:
feat_cols = [p_class, embedded_sex, bucket_age, fare, embedded_embarked]

In [250]:
x_data = df.drop('Survived',axis=1)

In [251]:
x_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,0.014151,S
1,1,female,38.0,1,0,0.139136,C
2,3,female,26.0,0,0,0.015469,S
3,1,female,35.0,1,0,0.103644,S
4,3,male,35.0,0,0,0.015713,S


In [252]:
labels = df['Survived']

In [253]:
labels.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [254]:
from sklearn.model_selection import train_test_split

In [255]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=.3)

In [256]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,
                                                 batch_size=8,num_epochs=1000,
                                                 shuffle=True)

In [257]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[7,7,7,7],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\karab\\AppData\\Local\\Temp\\tmpicz859bz', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_tf_random_seed': 1}


In [258]:
dnn_model.train(input_fn=input_func,steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\karab\AppData\Local\Temp\tmpicz859bz\model.ckpt.
INFO:tensorflow:step = 1, loss = 5.61088
INFO:tensorflow:global_step/sec: 198.396
INFO:tensorflow:step = 101, loss = 4.02473 (0.516 sec)
INFO:tensorflow:global_step/sec: 289.298
INFO:tensorflow:step = 201, loss = 6.50461 (0.346 sec)
INFO:tensorflow:global_step/sec: 305.443
INFO:tensorflow:step = 301, loss = 4.71236 (0.325 sec)
INFO:tensorflow:global_step/sec: 268.875
INFO:tensorflow:step = 401, loss = 5.18241 (0.358 sec)
INFO:tensorflow:global_step/sec: 285.769
INFO:tensorflow:step = 501, loss = 4.867 (0.365 sec)
INFO:tensorflow:global_step/sec: 299.936
INFO:tensorflow:step = 601, loss = 4.389 (0.328 sec)
INFO:tensorflow:global_step/sec: 318.19
INFO:tensorflow:step = 701, loss = 3.75292 (0.322 sec)
INFO:tensorflow:global_step/sec: 311.812
INFO:tensorflow:step = 801, loss = 4.57261 (0.321 sec)
INFO:tensorflow:global_step/sec: 284.911
INFO:te

INFO:tensorflow:global_step/sec: 338.653
INFO:tensorflow:step = 8401, loss = 4.34572 (0.302 sec)
INFO:tensorflow:global_step/sec: 278.645
INFO:tensorflow:step = 8501, loss = 2.12867 (0.357 sec)
INFO:tensorflow:global_step/sec: 250.367
INFO:tensorflow:step = 8601, loss = 1.37006 (0.389 sec)
INFO:tensorflow:global_step/sec: 292.976
INFO:tensorflow:step = 8701, loss = 5.86888 (0.349 sec)
INFO:tensorflow:global_step/sec: 234.409
INFO:tensorflow:step = 8801, loss = 2.03047 (0.425 sec)
INFO:tensorflow:global_step/sec: 289.908
INFO:tensorflow:step = 8901, loss = 1.48285 (0.346 sec)
INFO:tensorflow:global_step/sec: 277.24
INFO:tensorflow:step = 9001, loss = 3.76324 (0.358 sec)
INFO:tensorflow:global_step/sec: 288.778
INFO:tensorflow:step = 9101, loss = 1.92462 (0.352 sec)
INFO:tensorflow:global_step/sec: 283.478
INFO:tensorflow:step = 9201, loss = 3.81673 (0.357 sec)
INFO:tensorflow:global_step/sec: 280.935
INFO:tensorflow:step = 9301, loss = 1.65372 (0.349 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 295.617
INFO:tensorflow:step = 16801, loss = 5.17391 (0.331 sec)
INFO:tensorflow:global_step/sec: 299.268
INFO:tensorflow:step = 16901, loss = 5.15104 (0.340 sec)
INFO:tensorflow:global_step/sec: 295.083
INFO:tensorflow:step = 17001, loss = 5.43833 (0.336 sec)
INFO:tensorflow:global_step/sec: 294.37
INFO:tensorflow:step = 17101, loss = 1.14939 (0.341 sec)
INFO:tensorflow:global_step/sec: 288.608
INFO:tensorflow:step = 17201, loss = 6.8891 (0.341 sec)
INFO:tensorflow:global_step/sec: 293.578
INFO:tensorflow:step = 17301, loss = 3.68037 (0.344 sec)
INFO:tensorflow:global_step/sec: 279.621
INFO:tensorflow:step = 17401, loss = 0.791854 (0.362 sec)
INFO:tensorflow:global_step/sec: 306.627
INFO:tensorflow:step = 17501, loss = 1.64311 (0.326 sec)
INFO:tensorflow:global_step/sec: 306.425
INFO:tensorflow:step = 17601, loss = 2.01388 (0.333 sec)
INFO:tensorflow:global_step/sec: 265.193
INFO:tensorflow:step = 17701, loss = 6.55904 (0.371 sec)
INFO:tensorflow:globa

In [259]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,
                                                      batch_size=10,num_epochs=1,
                                                      shuffle=False)

In [260]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2018-05-29-01:23:17
INFO:tensorflow:Restoring parameters from C:\Users\karab\AppData\Local\Temp\tmpicz859bz\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-05-29-01:23:18
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.813084, accuracy_baseline = 0.598131, auc = 0.837845, auc_precision_recall = 0.828026, average_loss = 0.471348, global_step = 20000, label/mean = 0.401869, loss = 4.58493, prediction/mean = 0.401396


{'accuracy': 0.81308413,
 'accuracy_baseline': 0.59813082,
 'auc': 0.83784521,
 'auc_precision_recall': 0.82802594,
 'average_loss': 0.47134796,
 'global_step': 20000,
 'label/mean': 0.40186915,
 'loss': 4.5849299,
 'prediction/mean': 0.40139574}